# Processing Logs Explorer

Quick analysis of InfoTransform processing logs from SQLite database.

In [3]:
import sqlite3
import pandas as pd
from pathlib import Path

# Database path
DB_PATH = Path('/data/processing_logs.db')

def query_db(sql, params=None):
    """Execute query and return DataFrame"""
    conn = sqlite3.connect(str(DB_PATH))
    df = pd.read_sql_query(sql, conn, params=params)
    conn.close()
    return df

print(f"✅ Database: {DB_PATH.absolute()}")
print(f"✅ Exists: {DB_PATH.exists()}")
if DB_PATH.exists():
    print(f"✅ Size: {DB_PATH.stat().st_size / 1024:.2f} KB")

✅ Database: /data/processing_logs.db
✅ Exists: False


## Overall Statistics

In [2]:
# Summary stats
summary = query_db("""
    SELECT 
        COUNT(*) as total_runs,
        SUM(total_files) as total_files,
        SUM(successful_files) as successful,
        SUM(failed_files) as failed,
        SUM(total_tokens) as total_tokens,
        AVG(duration_seconds) as avg_duration
    FROM processing_runs
    WHERE status = 'completed'
""")

summary

OperationalError: unable to open database file

## Recent Runs

In [ ]:
# Last 10 runs
recent = query_db("""
    SELECT 
        run_id,
        start_timestamp,
        model_key,
        total_files,
        successful_files,
        total_tokens,
        duration_seconds,
        status
    FROM processing_runs
    ORDER BY start_timestamp DESC
    LIMIT 10
""")

recent

## Token Usage by Model

In [ ]:
# Token usage per model
tokens_by_model = query_db("""
    SELECT 
        model_key,
        COUNT(*) as runs,
        SUM(total_tokens) as total_tokens,
        AVG(total_tokens) as avg_tokens,
        SUM(input_tokens) as input_tokens,
        SUM(output_tokens) as output_tokens
    FROM processing_runs
    WHERE status = 'completed'
    GROUP BY model_key
    ORDER BY total_tokens DESC
""")

tokens_by_model

## Performance Metrics

In [ ]:
# Performance by model
performance = query_db("""
    SELECT 
        model_key,
        COUNT(*) as runs,
        AVG(duration_seconds) as avg_duration,
        AVG(total_files) as avg_files,
        ROUND(AVG(CAST(successful_files AS FLOAT) / NULLIF(total_files, 0) * 100), 2) as success_rate
    FROM processing_runs
    WHERE status = 'completed'
    GROUP BY model_key
""")

performance

## Daily Statistics

In [ ]:
# Last 7 days
daily = query_db("""
    SELECT 
        DATE(start_timestamp) as date,
        COUNT(*) as runs,
        SUM(total_files) as files,
        SUM(total_tokens) as tokens
    FROM processing_runs
    WHERE status = 'completed'
      AND start_timestamp >= datetime('now', '-7 days')
    GROUP BY DATE(start_timestamp)
    ORDER BY date DESC
""")

daily

## Cost Estimation

In [ ]:
# Cost estimate (update pricing as needed)
INPUT_PRICE_PER_1M = 0.15   # $0.15 per 1M input tokens
OUTPUT_PRICE_PER_1M = 0.60  # $0.60 per 1M output tokens

cost_df = tokens_by_model.copy()
cost_df['input_cost'] = (cost_df['input_tokens'] / 1_000_000) * INPUT_PRICE_PER_1M
cost_df['output_cost'] = (cost_df['output_tokens'] / 1_000_000) * OUTPUT_PRICE_PER_1M
cost_df['total_cost'] = cost_df['input_cost'] + cost_df['output_cost']

print(f"Total Estimated Cost: ${cost_df['total_cost'].sum():.4f}")
cost_df[['model_key', 'runs', 'total_tokens', 'total_cost']]

## Export Data

In [ ]:
# Export all completed runs to CSV
all_runs = query_db("SELECT * FROM processing_runs WHERE status = 'completed' ORDER BY start_timestamp DESC")
all_runs.to_csv('processing_logs_export.csv', index=False)
print(f"✅ Exported {len(all_runs)} runs to processing_logs_export.csv")